In [1]:
# Copyright (c) 2023-2024 Pietro Luca Mazzacuva <pietroluca.mazzacuva@unicampus.it>
#
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.

import pandas as pd
import numpy as np
import os, sys, pysam, gzip, subprocess, shlex, time
from datetime import datetime
from tqdm import tqdm
from multiprocessing import Pool
from sklearn.preprocessing import OneHotEncoder

def give_inputs(cell_line):
    
    files_path = "/lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/independent_datasets/{}".format(cells)
    
    if cell_line == "HEK293T":  
        samples = [["outTable_599710609", "outTable_905657585", "outTable_208420383"],
                   ["outTable_572868058", "outTable_364841872", "outTable_814257267"],
                   ["outTable_110067244", "outTable_597789462", "outTable_530905096"]]

        rmsk_file = "rmsk_hg38.sorted.gtf.gz"
        refseq_file = "hg38.110.ncbiRefSeq.sorted.gtf.gz"

    elif cell_line == "HEK":
        samples = [["outTable_724242056", "outTable_816573740"],
                   ["outTable_580067564", "outTable_718392497"],
                   ["outTable_181728208", "outTable_854894021"]]
        rmsk_file = "rmsk.sorted.gtf.gz"
        refseq_file = "hg19.ncbiRefSeq.sorted.gtf.gz"

    else:
        samples = [["outTable_192318299", "outTable_436061877"],
                   ["outTable_535670354", "outTable_396704193"],
                   ["outTable_773331943", "outTable_302610513"]]
        rmsk_file = "rmsk.sorted.gtf.gz"
        refseq_file = "hg19.ncbiRefSeq.sorted.gtf.gz"
    
    return samples, rmsk_file, refseq_file, files_path

def extraction(prefix,  AG_min, AGfreq_threshold, cov_threshold, interval):    
    
    starttime = datetime.now()

    editing = []
    with gzip.open(prefix+".gz") as redi:
        for c,l in enumerate(redi):
            line = l.decode("utf-8").rstrip().split("\t")
            if line[0].find("chr") != -1:
                if line[4] != "-":
                    if int(line[4]) >= cov_threshold:
                        if line[2] == "A":
                            if line[7] == "AG":    
                                AG_rna = eval(line[6])[2]/sum(eval(line[6]))
                                if AG_rna >= AGfreq_threshold:
                                    if eval(line[6])[2] >= AG_min:
                                        editing.append(line)
                          
            if c % 50000000 == 0:
                print(f"\tSites evaluated: {c}")
    print("Total evaluated rows:", c)
    editing = pd.DataFrame(editing)
    print("Total extracted Candidates Editing sites for current sample:", editing.shape[0])
    stoptime = datetime.now()
    print(f"[{datetime.now()}] Extraction of Editing Candidates finished for current sample. Elapsed time: {stoptime-starttime}.")
    columns = ["Region", "Position", "Ref", "Strand", "Cov", "Qual", "Bases", "AllSubs", "Freq", "gCov", "gQual", "g[A,C,G,T]", "gAllSubs", "gFreq"]
    editing.columns = columns
    print(f"[{datetime.now()}] Starting extraction of intervals.")
    ohe = OneHotEncoder()
    ohe.fit(np.array(["A", "C", "G", "T"]).reshape(-1, 1))

    intervals = []
    starttime_preds = datetime.now()
    total_extracted = 0
    features_extracted_filepath = prefix+ "_feature_vectors.tsv"
    features_extracted = open(features_extracted_filepath, "w")

    df = editing.query("Region != 'chrM'")
    print(f"[{datetime.now()}] Loading reditable with tabix and pysam:", prefix)
    start_time = datetime.now()
    srr = pysam.TabixFile(prefix+".gz")
    with tqdm(total=df.shape[0], position=0, leave=True) as pbar:
        for site in df.itertuples():
            start = int(site.Position) - ((interval-1)/2)
            stop = int(site.Position) + ((interval-1)/2)
            AGrna = eval(site.Bases)[2]/sum(eval(site.Bases))
            srr_interval = []
            for s in srr.fetch(site.Region, start-1, stop):
                srr_interval.append(s.split("\t"))
            srr_interval = pd.DataFrame(srr_interval, columns=columns)
            if srr_interval.shape[0] == interval and len(set(srr_interval["Strand"])) == 1:
                intervals.append([site.Region, site.Position, site.Ref, site.Strand, AGrna, site.Bases, start, stop, stop-start + 1, srr_interval.shape[0]])
                total_extracted += 1
                strand = site.Strand
                seq = srr_interval.Ref.values.reshape(-1,1)
                seq_ohe = ohe.transform(seq).toarray().T
                vects_freqs = []
                strands = []
                vects = []
                for vect in srr_interval["Bases"]:
                    vect = np.array(eval(vect))
                    cov = sum(vect)
                    vect_freqs = vect / cov
                    vects_freqs.append(vect_freqs)
                    vects.append(vect)
                vects_freqs = np.array(vects_freqs).T
                vects = np.array(vects).T
                encoded_site = pd.concat([pd.DataFrame(seq_ohe), pd.DataFrame(vects_freqs)])
                encoded_site.reset_index(drop=True, inplace=True)
                if strand == 0: 
                    encoded_site = pd.DataFrame(np.flip(encoded_site.values, axis=1))
                encoded_site.to_csv(features_extracted, mode="a", sep="\t", header = None, index=None)
            pbar.update(1)
    intervals = pd.DataFrame(intervals)
    print(f"[{datetime.now()}] Total extracted Editing sites: {total_extracted}.")
    stop_time_global = datetime.now()
    print(f"[{datetime.now()}] Features Extraction Finished. Elapsed time {datetime.now()-starttime_preds}.")
    features_extracted.close()
    
    intervals.columns = ["Region", "Position", "RefBase", "Strand", "FreqAGrna", "BasesCounts", "Start", "Stop", "Intlen", "TabixLen"]
    intervals.to_csv(prefix + "_intervals.tsv", sep="\t", index=None)
    print(f"[{datetime.now()}] Computation Finished. Total Elapsed time: {datetime.now()-starttime}")

def candidates_bona_fide_extraction(name, path, cells, AG_min, AGfreq_threshold, cov_threshold, rna_cov_threshold, gen):    

    starttime = datetime.now()
    sites = []
    
    wgs = pysam.TabixFile(f"{path}/{cells}_WGS.gz")                                                                  
    with gzip.open(f"{path}/{name}.gz") as redi:
        for c,l in enumerate(redi):
            line = l.decode("utf-8").rstrip().split("\t")
            if line[0].find("chr") != -1:
                if line[0] != "chrM":
                    if line[4] != "-":
                        if int(line[4]) >= rna_cov_threshold:
                            if line[2] == "A":
                                if "AG" in line[7] == "AG":    
                                    AG_rna = eval(line[6])[2]/sum(eval(line[6]))
                                    if AG_rna >= AGfreq_threshold:
                                        if eval(line[6])[2] >= AG_min:
                                            region = line[0]
                                            start = int(line[1])-1
                                            stop = int(line[1])
                                            for ROW_WGS in wgs.fetch(region, start, stop):
                                                row_wgs = ROW_WGS.split("\t")
                                                if row_wgs[9] !=  "-":
                                                    if int(row_wgs[9])>=cov_threshold:
                                                        if "AG" in row_wgs[12]:
                                                            sites.append([line[0], line[1], 0])
                                                        else:
                                                            if row_wgs[12] == "-":
                                                                if line[7] == "AG":
                                                                    sites.append([line[0], line[1], 1]) 
                                else:
                                    if  line[7] == "-":
                                        if gen != "wt" and gen != "oe":                                        
                                            sites.append([line[0], line[1], 2])
                                                                                           
            if c % 50000000 == 0:
                print(f"\tSites evaluated: {c}")
     
    sites = pd.DataFrame(sites)
    sites.columns = ["Region", "Position", "Class"]
    print("Total evaluated rows:", c)
    print(f"Total extracted candidates bona fide Sites for {name} sample: {sites.shape[0]}")
    stoptime = datetime.now()
    
    print(f"[{datetime.now()}] Extraction of candidates bonafide sites finished for {name} sample. Elapsed time: {stoptime-starttime}.")
    sites.to_csv(f"{path}/{name}_bona_fide_sites.tsv", sep="\t", index=None)
                                                                                 
def bonafide_identification(tables, path, rmsk, refseq):
                           
    u_path = "/lustrehome/pietrolucamazzacuva/filezilla-recas/scripts/utilities"                                                                            
                                                                                 
    if cells != "HEK293T":
        for i in range(3):
                bonafide_wt = pd.read_table(f"{path}/{tables[i][0]}_bona_fide_sites.tsv")
                bonafide_adar_inactive = pd.read_table(f"{path}/{tables[i][1]}_bona_fide_sites.tsv")
                bonafide_adar_inactive = bonafide_adar_inactive[bonafide_adar_inactive.loc[:, "Class"]!=2]
                bonafide = pd.concat([bonafide_wt, bonafide_adar_inactive], axis=0)
                bonafide.drop_duplicates(subset=["Region", "Position", "Class"], keep="first", inplace=True)
                bonafide.to_csv(f"{path}/{tables[i][0]}_{tables[i][1]}_candidates_bona_fide_sites.tsv", sep="\t", index=None, header=False)
                
    else:
        for i in range(3):
            for j in [0, 2]:
                bonafide_wt = pd.read_table(f"{path}/{tables[i][j]}_bona_fide_sites.tsv")
                bonafide_adar_inactive = pd.read_table(f"{path}/{tables[i][1]}_bona_fide_sites.tsv")
                bonafide_adar_inactive = bonafide_adar_inactive[bonafide_adar_inactive.loc[:, "Class"]!=2]
                bonafide = pd.concat([bonafide_wt, bonafide_adar_inactive], axis=0)
                bonafide.drop_duplicates(subset=["Region", "Position", "Class"], keep="first", inplace=True)
                bonafide.to_csv(f"{path}/{tables[i][j]}_{tables[i][1]}_candidates_bona_fide_sites.tsv", sep="\t", index=None, header=False)

    for file_name in os.listdir(path):
        if file_name.find("_candidates_bona_fide_sites.tsv") !=-1:
            df = pd.read_csv(os.path.join(path, file_name), sep="\t")
            df.to_csv(os.path.join(path, file_name), sep="\t", index=None, header=False)
            name = file_name.replace(".tsv", "")
            cmd_sh = "python3 {}/AnnotateTablePython3.py -a {}/{} -n rmsk -i {}/{}.tsv -o {}/{}.out.rmsk -u".format(u_path, u_path, rmsk, path, name, path, name)
            args = shlex.split(cmd_sh)
            p = subprocess.Popen(args, env=dict(os.environ, PATH="/lustrehome/pietrolucamazzacuva/anaconda3/envs/tensorflow-gpu/bin"))

    time.sleep(60)

    for name in os.listdir(path):
        if name.find("rmsk") != -1:
            cmd_sh = "python3 {}/AnnotateTablePython3.py -a {}/{} -i {}/{} -o {}/{}.refseq -u".format(u_path, u_path, refseq, path, name, path, name)
            args = shlex.split(cmd_sh)
            p = subprocess.Popen(args, env=dict(os.environ, PATH="/lustrehome/pietrolucamazzacuva/anaconda3/envs/tensorflow-gpu/bin"))

    time.sleep(60)

    cols = ["Region", "Position", "Class", "RMSK-Rep", "RMSK-Reg", "RefSeq-Rep", "RefSeq-Reg"]
    for file_name in os.listdir(path):
        if file_name.find(".refseq") !=-1:
            df = pd.read_table(os.path.join(path, file_name), header=None)
            name = file_name.replace(".out.rmsk.refseq", "_annoted.tsv")
            df.columns = cols
            df.to_csv(os.path.join(path, name), sep="\t", index=None)

    for file_name in os.listdir(path):
        if file_name.find("candidates_bona_fide_sites_annoted.tsv") !=-1:
            bona_fide = pd.read_csv(os.path.join(path, file_name), sep="\t")
            rep = bona_fide[(bona_fide.iloc[:, 3] != "-") & (bona_fide.iloc[:, 4] != "-")]
            non_rep = bona_fide[(bona_fide.iloc[:, 3] == "-") & (bona_fide.iloc[:, 4] == "-")]

            del bona_fide

            non_rep_n = non_rep[non_rep.iloc[:, 2]==0]
            non_rep_p = non_rep[non_rep.iloc[:, 2]==1]

            del non_rep

            non_rep_p = non_rep_p[(non_rep_p.iloc[:, 5] != "-") & (non_rep_p.iloc[:, 6] != "-")]
            bona_fide = pd.concat([rep, non_rep_n, non_rep_p])

            del rep, non_rep_n, non_rep_p

            name = file_name.replace("candidates_bona_fide_sites_annoted.tsv", "bona_fide_sites.tsv")
            bona_fide = bona_fide.sort_values(["Region", "Position"])
            bona_fide.to_csv(os.path.join(path, name), sep="\t", index=None)

In [2]:
min_dna_cov = 10
min_rna_cov = 50
min_AG_rate = 0.01
min_G = 3
seq_lenght = 101

cells = "a549"

samples, rmsk_file_name, refseq_file_name, filespath = give_inputs(cells)
    
inputs = []
for i in range(len(samples)):
    for j in range(len(samples[i])):
        inputs.append([os.path.join(filespath, f"{samples[i][j]}"), min_G, min_AG_rate, min_rna_cov, seq_lenght])

with Pool(9) as pool:
    pool.starmap(extraction, inputs)

inputs = []
for i in range(3):
    inputs.append([samples[i][0], filespath, cells, min_G, min_AG_rate, min_dna_cov, min_rna_cov, "wt"])
    inputs.append([samples[i][1], filespath, cells, min_G, min_AG_rate, min_dna_cov, min_rna_cov, "si"])
    
with Pool(6) as pool:
    pool.starmap(candidates_bona_fide_extraction, inputs)     

bonafide_identification(samples, filespath, rmsk_file_name, refseq_file_name)

	Sites evaluated: 0
	Sites evaluated: 0	Sites evaluated: 0	Sites evaluated: 0
	Sites evaluated: 0


	Sites evaluated: 0
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 200000000
	Sites evaluated: 200000000
	Sites evaluated: 200000000
	Sites evaluated: 200000000
	Sites evaluated: 200000000
	Sites evaluated: 200000000
Total evaluated rows: 216398797
Total extracted Candidates Editing sites for current sample: 6334
[2024-06-20 22:58:22.589626] Extraction of Editing Candidates finished for current sample. Elapsed time: 0:06:48.816

100%|██████████| 6332/6332 [00:20<00:00, 308.56it/s]


[2024-06-20 22:58:43.237277] Total extracted Editing sites: 2686.
[2024-06-20 22:58:43.238624] Features Extraction Finished. Elapsed time 0:00:20.645196.
[2024-06-20 22:58:43.255616] Computation Finished. Total Elapsed time: 0:07:09.482184
Total evaluated rows: 236217106
Total extracted Candidates Editing sites for current sample: 15176
[2024-06-20 22:58:59.608335] Extraction of Editing Candidates finished for current sample. Elapsed time: 0:07:25.834205.
[2024-06-20 22:58:59.610056] Starting extraction of intervals.
[2024-06-20 22:58:59.621243] Loading reditable with tabix and pysam: /lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/independent_datasets/a549/outTable_773331943


  0%|          | 0/15175 [00:00<?, ?it/s]

Total evaluated rows:

...

100%|██████████| 16326/16326 [00:49<00:00, 328.17it/s]


[2024-06-20 22:59:49.782611] Total extracted Editing sites: 6856.
[2024-06-20 22:59:49.784150] Features Extraction Finished. Elapsed time 0:00:49.909293.
[2024-06-20 22:59:49.823810] Computation Finished. Total Elapsed time: 0:08:16.050076


100%|██████████| 18396/18396 [00:56<00:00, 327.20it/s]


[2024-06-20 23:00:10.258783] Total extracted Editing sites: 7691.
[2024-06-20 23:00:10.260284] Features Extraction Finished. Elapsed time 0:00:56.317654.
[2024-06-20 23:00:10.299339] Computation Finished. Total Elapsed time: 0:08:36.525977
	Sites evaluated: 0
	Sites evaluated: 0
	Sites evaluated: 0
	Sites evaluated: 0
	Sites evaluated: 0
	Sites evaluated: 0
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 200000000
	Sites evaluated: 200000000
	Sites evaluated: 200000000
	Sites evaluated: 200000000
	Sites evaluated: 200000000
	S

Pysam version used: 0.22.0
Script time --> START: 20/06/2024 23:08:45
Table saved on /lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/independent_datasets/a549/outTable_192318299_outTable_436061877_candidates_bona_fide_sites.out.rmsk
Script time --> END: 20/06/2024 23:08:50
Pysam version used: 0.22.0
Script time --> START: 20/06/2024 23:08:45
Table saved on /lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/independent_datasets/a549/outTable_535670354_outTable_396704193_candidates_bona_fide_sites.out.rmsk
Script time --> END: 20/06/2024 23:08:50
Pysam version used: 0.22.0
Script time --> START: 20/06/2024 23:08:45
Table saved on /lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/independent_datasets/a549/outTable_773331943_outTable_302610513_candidates_bona_fide_sites.out.rmsk
Script time --> END: 20/06/2024 23:08:50
Pysam version used: 0.22.0
Script time --> START: 20/06/2024 23:09:43
Table saved on /lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/independent_d

In [3]:
min_dna_cov = 10
min_rna_cov = 50
min_AG_rate = 0.01
min_G = 3
seq_lenght = 101

cells = "HEK293T"

samples, rmsk_file_name, refseq_file_name, filespath = give_inputs(cells)
    
inputs = []
for i in range(len(samples)):
    for j in range(len(samples[i])):
        inputs.append([os.path.join(filespath, f"{samples[i][j]}"), min_G, min_AG_rate, min_rna_cov, seq_lenght])

with Pool(9) as pool:
    pool.starmap(extraction, inputs)

inputs = []
for i in range(3):
    inputs.append([samples[i][0], filespath, cells, min_G, min_AG_rate, min_dna_cov, min_rna_cov, "wt"])
    inputs.append([samples[i][1], filespath, cells, min_G, min_AG_rate, min_dna_cov, min_rna_cov, "ko"])
    inputs.append([samples[i][2], filespath, cells, min_G, min_AG_rate, min_dna_cov, min_rna_cov, "oe"])
    
with Pool(6) as pool:
    pool.starmap(candidates_bona_fide_extraction, inputs)     

bonafide_identification(samples, filespath, rmsk_file_name, refseq_file_name)

	Sites evaluated: 0	Sites evaluated: 0

	Sites evaluated: 0
	Sites evaluated: 0
	Sites evaluated: 0
	Sites evaluated: 0
	Sites evaluated: 0	Sites evaluated: 0

	Sites evaluated: 0
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
Total evaluated rows: 194677901
Total extracted Candidates Editing sites 

 20%|█▉        | 2299/11701 [00:07<00:28, 333.76it/s]

Total evaluated rows: 199831579
Total extracted Candidates Editing sites for current sample: 24397
[2024-06-20 23:16:57.528219] Extraction of Editing Candidates finished for current sample. Elapsed time: 0:06:13.991545.
[2024-06-20 23:16:57.529895] Starting extraction of intervals.
[2024-06-20 23:16:57.547187] Loading reditable with tabix and pysam: /lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/independent_datasets/HEK293T/outTable_110067244


 21%|██▏       | 2502/11701 [00:08<00:28, 325.41it/s]

	Sites evaluated: 200000000


...

 42%|████▏     | 22813/54968 [01:20<01:39, 322.71it/s]

[2024-06-20 23:23:51.235882] Computation Finished. Total Elapsed time: 0:13:07.699063


100%|██████████| 54968/54968 [03:10<00:00, 288.38it/s]


[2024-06-20 23:25:41.172069] Total extracted Editing sites: 22284.
[2024-06-20 23:25:41.173951] Features Extraction Finished. Elapsed time 0:03:10.724731.
[2024-06-20 23:25:41.274966] Computation Finished. Total Elapsed time: 0:14:57.739403
	Sites evaluated: 0
	Sites evaluated: 0
	Sites evaluated: 0
	Sites evaluated: 0
	Sites evaluated: 0
	Sites evaluated: 0
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
Total evaluated rows: 194677901
Total extracted candidates bona fide Sites for outTable_905657585 sample: 8210567
[2024-06-20 23:32:33.835527

Pysam version used: 0.22.0
Script time --> START: 20/06/2024 23:48:06
Table saved on /lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/independent_datasets/HEK293T/outTable_110067244_outTable_597789462_candidates_bona_fide_sites.out.rmsk
Script time --> END: 20/06/2024 23:48:14
Pysam version used: 0.22.0
Script time --> START: 20/06/2024 23:48:06
Table saved on /lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/independent_datasets/HEK293T/outTable_572868058_outTable_364841872_candidates_bona_fide_sites.out.rmsk
Script time --> END: 20/06/2024 23:48:15
Pysam version used: 0.22.0
Script time --> START: 20/06/2024 23:48:06
Table saved on /lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/independent_datasets/HEK293T/outTable_208420383_outTable_905657585_candidates_bona_fide_sites.out.rmsk
Script time --> END: 20/06/2024 23:48:17
Pysam version used: 0.22.0
Script time --> START: 20/06/2024 23:48:06
Table saved on /lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/inde

In [4]:
min_dna_cov = 10
min_rna_cov = 30
min_AG_rate = 0.01
min_G = 2
seq_lenght = 101

cells = "HEK"

samples, rmsk_file_name, refseq_file_name, filespath = give_inputs(cells)
    
inputs = []
for i in range(len(samples)):
    for j in range(len(samples[i])):
        inputs.append([os.path.join(filespath, f"{samples[i][j]}"), min_G, min_AG_rate, min_rna_cov, seq_lenght])

with Pool(9) as pool:
    pool.starmap(extraction, inputs)
    
inputs = []
for i in range(3):
    inputs.append([samples[i][0], filespath, cells, min_G, min_AG_rate, min_dna_cov, min_rna_cov, "wt"])
    inputs.append([samples[i][1], filespath, cells, min_G, min_AG_rate, min_dna_cov, min_rna_cov, "ko"])

with Pool(6) as pool:
    pool.starmap(candidates_bona_fide_extraction, inputs)     

bonafide_identification(samples, filespath, rmsk_file_name, refseq_file_name)

	Sites evaluated: 0
	Sites evaluated: 0	Sites evaluated: 0
	Sites evaluated: 0

	Sites evaluated: 0
	Sites evaluated: 0
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 200000000
	Sites evaluated: 200000000
	Sites evaluated: 200000000
	Sites evaluated: 200000000
	Sites evaluated: 200000000
	Sites evaluated: 200000000
	Sites evaluated: 250000000
	Sites evaluated: 250000000
	Sites evaluated: 250000000
	Sites evaluated: 250000000
	Sites evaluated: 250000000
	Sites evaluated: 250000000
	Sites evaluated: 300000000
	Sites evaluated: 

 27%|██▋       | 1340/4980 [00:15<00:28, 125.91it/s]

Total evaluated rows: 422584480
Total extracted Candidates Editing sites for current sample: 12583
[2024-06-21 00:03:09.575480] Extraction of Editing Candidates finished for current sample. Elapsed time: 0:13:01.698895.
[2024-06-21 00:03:09.577170] Starting extraction of intervals.
[2024-06-21 00:03:09.593031] Loading reditable with tabix and pysam: /lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/independent_datasets/HEK/outTable_181728208


100%|██████████| 4980/4980 [00:35<00:00, 139.97it/s]]


[2024-06-21 00:03:29.619463] Total extracted Editing sites: 2111.
[2024-06-21 00:03:29.620904] Features Extraction Finished. Elapsed time 0:00:35.730531.
[2024-06-21 00:03:29.634804] Computation Finished. Total Elapsed time: 0:13:21.758437


...

100%|██████████| 12180/12180 [00:56<00:00, 216.97it/s]


[2024-06-21 00:05:48.038639] Total extracted Editing sites: 5312.
[2024-06-21 00:05:48.040024] Features Extraction Finished. Elapsed time 0:00:56.275927.
[2024-06-21 00:05:48.068157] Computation Finished. Total Elapsed time: 0:15:40.191933
Total evaluated rows: 519011879
Total extracted Candidates Editing sites for current sample: 7987
[2024-06-21 00:06:14.468222] Extraction of Editing Candidates finished for current sample. Elapsed time: 0:16:06.591532.
[2024-06-21 00:06:14.469949] Starting extraction of intervals.
[2024-06-21 00:06:14.478656] Loading reditable with tabix and pysam: /lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/independent_datasets/HEK/outTable_854894021


100%|██████████| 7971/7971 [00:46<00:00, 173.10it/s]


[2024-06-21 00:07:00.883094] Total extracted Editing sites: 3620.
[2024-06-21 00:07:00.884411] Features Extraction Finished. Elapsed time 0:00:46.412478.
[2024-06-21 00:07:00.903324] Computation Finished. Total Elapsed time: 0:16:53.026649
	Sites evaluated: 0
	Sites evaluated: 0
	Sites evaluated: 0
	Sites evaluated: 0
	Sites evaluated: 0
	Sites evaluated: 0
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 200000000
	Sites evaluated: 200000000
	Sites evaluated: 200000000
	Sites evaluated: 200000000
	Sites evaluated: 200000000
	S

Pysam version used: 0.22.0
Script time --> START: 21/06/2024 00:24:28
Table saved on /lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/independent_datasets/HEK/outTable_580067564_outTable_718392497_candidates_bona_fide_sites.out.rmsk
Script time --> END: 21/06/2024 00:24:32
Pysam version used: 0.22.0
Script time --> START: 21/06/2024 00:24:28
Table saved on /lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/independent_datasets/HEK/outTable_724242056_outTable_816573740_candidates_bona_fide_sites.out.rmsk
Script time --> END: 21/06/2024 00:24:32
Pysam version used: 0.22.0
Script time --> START: 21/06/2024 00:24:28
Table saved on /lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/independent_datasets/HEK/outTable_181728208_outTable_854894021_candidates_bona_fide_sites.out.rmsk
Script time --> END: 21/06/2024 00:24:33
Pysam version used: 0.22.0
Script time --> START: 21/06/2024 00:25:26
Table saved on /lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/independent_data